In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ASNA-hackathon/sample_submission.csv
/kaggle/input/ASNA-hackathon/train.csv
/kaggle/input/ASNA-hackathon/test.csv


In [2]:
#Importing packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, auc, f1_score, roc_curve, roc_auc_score, classification_report, precision_recall_curve
from imblearn.ensemble import BalancedRandomForestClassifier
import xgboost
import lightgbm as lgb
from lightgbm import LGBMClassifier
from statistics import mean

In [3]:
df_train = pd.read_csv("/kaggle/input/ASNA-hackathon/train.csv") #7290
df_test = pd.read_csv("/kaggle/input/ASNA-hackathon/test.csv") #1844

In [4]:
df_train['Claim over 1k'].value_counts()

Claim over 1k
0    6458
1     832
Name: count, dtype: int64

In [5]:
df_train['Income'].replace(0, np.nan, inplace=True)
df_test['Income'].replace(0, np.nan, inplace=True)

/tmp/ipykernel_17/2961618207.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['Income'].replace(0, np.nan, inplace=True)
/tmp/ipykernel_17/2961618207.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [6]:
for c in df_train.columns:
    col_type = df_train[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df_train[c] = df_train[c].astype('category')
        
for c in df_test.columns:
    col_type = df_test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df_test[c] = df_test[c].astype('category')

In [7]:
cols_with_missing = [col for col in df_train.columns 
                                 if df_train[col].isnull().any()]                                  
candidate_train_predictors = df_train.drop(['CustomerID', 'Claim over 1k', 'Coverage Index', 'Education Index', 'Employment Status Index', 'Marital Status Index', 'Policy Type Index', 'Policy Index', 'Sales Channel Index', 'Vehicle Size Index'] + cols_with_missing, axis=1)
candidate_test_predictors = df_test.drop(['CustomerID', 'Coverage Index', 'Education Index', 'Employment Status Index', 'Marital Status Index', 'Policy Type Index', 'Policy Index', 'Sales Channel Index', 'Vehicle Size Index'] + cols_with_missing, axis=1)

# "cardinality" means the number of unique values in a column.
# We use it as our only way to select categorical columns here. This is convenient, though
# a little arbitrary.
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

In [8]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_training_predictors

,Customer Lifetime Value,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Renew Offer Type
0,3622.69,4,107,3,1,3
1,10610.21,26,14,0,7,1
2,13868.02,11,38,0,3,3
3,3119.69,16,32,0,1,2
4,5999.04,45,126,0,1,2
...,...,...,...,...,...,...
7285,27500.54,0,28,1,2,1
7286,11750.03,9,107,0,2,3
7287,7757.04,23,119,0,9,1
7288,3465.16,19,89,0,1,3


In [9]:
y = df_train['Claim over 1k']
X = one_hot_encoded_training_predictors
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

We will also try LightGBM classifier and see if there are any improvements.

In [10]:
y = df_train['Claim over 1k']
X = candidate_train_predictors
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.20, random_state=314, stratify=y)

In [11]:
lgbm = LGBMClassifier(objective='binary')
lgbm.fit(train_X,train_y)
val_y_predlgbm = lgbm.predict(val_X)

fpr, tpr, thresholds = metrics.roc_curve(val_y, val_y_predlgbm, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(val_y, val_y_predlgbm)
auc_pr = auc(recall, precision)

print("Recall of LightGBM on test set:",metrics.recall_score(val_y, val_y_predlgbm,average='binary')) 
print('Precision of LightGBM on test set: {:.2f}'.format(precision_score(val_y, val_y_predlgbm))) 
print('F1 score of LightGBM on test set: {:.2f}'.format(f1_score(val_y, val_y_predlgbm, average='weighted'))) # to account for disbalance

print(f"ROC-AUC of LightGBM on test set: {auc_roc}")
print(f"PR-AUC of LightGBM on test set: {auc_pr}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 666, number of negative: 5166
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 526
[LightGBM] [Info] Number of data points in the train set: 5832, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.114198 -> initscore=-2.048564
[LightGBM] [Info] Start training from score -2.048564
Recall of LightGBM on test set: 0.5
Precision of LightGBM on test set: 0.69
F1 score of LightGBM on test set: 0.91
ROC-AUC of LightGBM on test set: 0.7356811145510836
PR-AUC of LightGBM on test set: 0.6242969821673525


Resampling

In [12]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [13]:
undersample = RandomUnderSampler(sampling_strategy='majority')
# fit and apply the transform
train_X, train_y = undersample.fit_resample(X, y)
# Build the model with the random forest regression algorithm:
lgbm = LGBMClassifier(objective='binary')
lgbm.fit(train_X,train_y)
val_y_predlgbm = lgbm.predict(val_X)

fpr, tpr, thresholds = metrics.roc_curve(val_y, val_y_predlgbm, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(val_y, val_y_predlgbm)
auc_pr = auc(recall, precision)

print("Recall of LightGBM on test set:",metrics.recall_score(val_y, val_y_predlgbm,average='binary')) 
print('Precision of LightGBM on test set: {:.2f}'.format(precision_score(val_y, val_y_predlgbm))) 
print('F1 score of LightGBM on test set: {:.2f}'.format(f1_score(val_y, val_y_predlgbm, average='weighted'))) # to account for disbalance

print(f"ROC-AUC of LightGBM on test set: {auc_roc}")
print(f"PR-AUC of LightGBM on test set: {auc_pr}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 832, number of negative: 832
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000357 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 526
[LightGBM] [Info] Number of data points in the train set: 1664, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Recall of LightGBM on test set: 1.0
Precision of LightGBM on test set: 0.45
F1 score of LightGBM on test set: 0.88
ROC-AUC of LightGBM on test set: 0.9210526315789473
PR-AUC of LightGBM on test set: 0.7243243243243244


In [14]:
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
train_X, train_y = oversample.fit_resample(X, y)
# Build the model with the random forest regression algorithm:
lgbm = LGBMClassifier(objective='binary')
lgbm.fit(train_X,train_y)
val_y_predlgbm = lgbm.predict(val_X)

fpr, tpr, thresholds = metrics.roc_curve(val_y, val_y_predlgbm, pos_label=1)
auc_roc = metrics.auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(val_y, val_y_predlgbm)
auc_pr = auc(recall, precision)

print("Recall of LightGBM on test set:",metrics.recall_score(val_y, val_y_predlgbm,average='binary')) 
print('Precision of LightGBM on test set: {:.2f}'.format(precision_score(val_y, val_y_predlgbm))) 
print('F1 score of LightGBM on test set: {:.2f}'.format(f1_score(val_y, val_y_predlgbm, average='weighted'))) # to account for disbalance

print(f"ROC-AUC of LightGBM on test set: {auc_roc}")
print(f"PR-AUC of LightGBM on test set: {auc_pr}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 6458, number of negative: 6458
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 526
[LightGBM] [Info] Number of data points in the train set: 12916, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Recall of LightGBM on test set: 1.0
Precision of LightGBM on test set: 0.83
F1 score of LightGBM on test set: 0.98
ROC-AUC of LightGBM on test set: 0.9864551083591332
PR-AUC of LightGBM on test set: 0.9129353233830846


In [15]:
predictions = lgbm.predict(candidate_test_predictors)
output = pd.DataFrame({'CustomerID': df_test.CustomerID, 'Claim over 1k': predictions})
output.to_csv('submission.csv', index=False)

In [16]:
# over = RandomOverSampler(sampling_strategy=0.7)
# train_X, train_y = over.fit_resample(X, y)
# # define undersampling strategy
# under = RandomUnderSampler(sampling_strategy=0.5)
# # fit and apply the transform
# train_X, train_y = under.fit_resample(X, y)
# # summarize class distribution

# # Build the model with the random forest regression algorithm:
# lgbm = LGBMClassifier(objective='binary')
# lgbm.fit(train_X, train_y)
# val_y_predlgbm = lgbm.predict(val_X)

# fpr, tpr, thresholds = metrics.roc_curve(val_y, val_y_predlgbm, pos_label=1)
# auc_roc = metrics.auc(fpr, tpr)
# precision, recall, _ = precision_recall_curve(val_y, val_y_predlgbm)
# auc_pr = auc(recall, precision)

# print("Recall of LightGBM on test set:",metrics.recall_score(val_y, val_y_predlgbm,average='binary')) 
# print('Precision of LightGBM on test set: {:.2f}'.format(precision_score(val_y, val_y_predlgbm))) 
# print('F1 score of LightGBM on test set: {:.2f}'.format(f1_score(val_y, val_y_predlgbm, average='weighted'))) # to account for disbalance

# print(f"ROC-AUC of LightGBM on test set: {auc_roc}")
# print(f"PR-AUC of LightGBM on test set: {auc_pr}")

In [17]:
# # https://www.kaggle.com/code/mlisovyi/beware-of-categorical-features-in-lgbm
# fit_params={"eval_metric" : 'auc', 
#             "eval_set" : [(val_X,val_y)],
#             'eval_names': ['valid'],
#             'feature_name': 'auto', # that's actually the default
#             'categorical_feature': ['State', 'Response', 'Coverage', 'Education', 'Employment Status',
#        'Gender', 'Marital Status', 'Policy Type', 'Policy', 'Sales Channel',
#        'Vehicle Size', 'Renew Offer Type']
#            }

In [18]:
# #n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 1000 define only the absolute maximum
# clf = lgb.LGBMClassifier(num_leaves= 15, max_depth=-1, 
#                          random_state=314, 
#                          silent=True, 
#                          metric='None', 
#                          n_jobs=4, 
#                          n_estimators=1000,
#                          colsample_bytree=0.9,
#                          subsample=0.9,
#                          learning_rate=0.1)

In [19]:
# #force larger number of max trees and smaller learning rate
# clf.fit(train_X, train_y, **fit_params)

# val_y_predlgbm = lgbm.predict(val_X)

# fpr, tpr, thresholds = metrics.roc_curve(val_y, val_y_predlgbm, pos_label=1)
# auc_roc = metrics.auc(fpr, tpr)
# precision, recall, _ = precision_recall_curve(val_y, val_y_predlgbm)
# auc_pr = auc(recall, precision)

# print("Recall of LightGBM on test set:",metrics.recall_score(val_y, val_y_predlgbm,average='binary')) 
# print('Precision of LightGBM on test set: {:.2f}'.format(precision_score(val_y, val_y_predlgbm))) 
# print('F1 score of LightGBM on test set: {:.2f}'.format(f1_score(val_y, val_y_predlgbm, average='weighted'))) # to account for disbalance

# print(f"ROC-AUC of LightGBM on test set: {auc_roc}")
# print(f"PR-AUC of LightGBM on test set: {auc_pr}")

In [20]:
# predictions = clf.predict(candidate_test_predictors)
# output = pd.DataFrame({'CustomerID': df_test.CustomerID, 'Claim over 1k': predictions})
# output.to_csv('submission.csv', index=False)